# EMGdecomPy Worflow

This notebook allows for simple visualisation of decomposition results with via EMGDecomPy package. For simplicity, you may want to select the usual filepath and filename to store raw EMG signals, and simply "Run All Cells" each time you want to perform decomposition process.

## Imports - dependencies

Please run the cells below to add all the dependencies to run this notebook.   
First, import the proprietary open-source python packages that were used in building this application:

In [1]:
# Widgets for interactivity
import panel as pn # Panel allows binding functions to plots
pn.extension('vega')
from panel.interact import interact, interactive, fixed, interact_manual
from panel import widgets
from scipy.io import loadmat

# Pickle allows saving and loading python object into a file
import pickle

## Imports - EMGDecomPy

Now, import the proprietary EMGDecomPy package scripts for decomposing and visualizing the electromyography data:

In [2]:
from emgdecompy.decomposition import *
from emgdecompy.contrast import *
from emgdecompy.viz import *
from emgdecompy.preprocessing import *

## Load the raw EMG data

Please specify the filepath where the raw data is located. 

This raw data will be processed via an algorithm built after the process proposed by _Negro et al 2016_ paper. In order for the decomposition to accurately process the data, it needs to be in a clean format. File __extension__ must be __.mat__, the __data type__ must be __dictionary__, and the __key__ containing the raw electromyography signal to be decomposed must be named __'SIG'__.

A simplified example of raw data can be seen below:

Note the aforementioned __'SIG'__ key-value pair, containing array of arrays. For the purposes of decomposition, __'SIG'__ is the only key needed within the dictionary.

In [3]:
raw_data_dict = loadmat('../data/raw/gl_10.mat')

## Run decomposition

Run the decomposition on raw data to retrieve a dictionary containing decomposed results.

__There is a number of parameters you may choose to customize:__

| Parameter    | Data Type                    | Explanation                                                                                  |
|--------------|------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| x            | numpy.ndarray                | Raw EMG signal. This is expected inside the 'SIG' key within the raw signal loaded above.  
| discard      | slice, int, or array of ints | Indices of channels to discard. Useful if you are looking to only analyze some channels, or expect some channels to contain disruptive data.                               |
| R            | int                          | How far to extend x during the extension step of the algorithm.                                                                                                            |
| M            | int                          | Number of iterations to run decomposition for.                                                                                                                             |
| filter       | bool                         | Whether to band-pass filter the raw EMG signal or not.                                                                                                                     |
| lowcut       | float                        | Lower range of band-pass filter.                                                                                                                                           |
| highcut      | float                        | Upper range of band-pass filter.                                                                                                                                           |
| fs           | float                        | Sampling frequency in Hz.                                                                                                                                                  |
| order        | int                          | Order of band-pass filter.                                                                                                                                                 |
| peel         | bool                         | Whether to conduct "peel-off" process or not.                                                                                                                              |
| Tolx         | float                        | Tolerance for element-wise comparison in separation.                                                                                                                       |
| contrast_fun | function                     | Contrast function to use. skew, og_cosh or exp_sq                                                                                                                          |
| ortho_fun    | function                     | Orthogonalization function to use. gram_schmidt or deflate                                                                                                                 |
| max_iter_sep | int > 0                      | Maximum iterations for fixed point algorithm.                                                                                                                              |
| l            | int                          | Required minimal horizontal distance between peaks in peak-finding algorithm. Default value of 31 samples is approximately equivalent to 15 ms at a 2048 Hz sampling rate. |
| sil_pnr      | bool                         | Whether to use SIL or PNR as acceptance criterion. Default value of True uses SIL.                                                                                         |
| thresh       | float                        | SIL/PNR threshold for accepting a separation vector.                                                                                                                       |
| max_iter_ref | int > 0                      | Maximum iterations for refinement.                                                                                                                                         |
| random_seed  | int                          | Used to initialize the pseudo-random processes in the function.                                                                                                            |
| verbose      | bool                         | If true, decomposition information is printed.                                                                                                                             |


In [4]:
# output = decomposition(
#     raw_data_dict["SIG"],
#     discard=5,
#     R=16,
#     M=64,
#     filter=True,
#     lowcut=10,
#     highcut = 900,
#     fs=2048,
#     order=6,
#     peel=False,
#     Tolx=10e-4,
#     contrast_fun=skew,
#     ortho_fun=gram_schmidt,
#     max_iter_sep=10,
#     l=31,
#     sil_pnr=True,
#     thresh=0.9,
#     max_iter_ref=10,
#     random_seed=None,
#     verbose=False
# )

The output of decomposition is a dictionary containing the following keys:

__'B'__: Matrix whose columns contain the accepted separation vectors.

__'MUPulses'__: Firing indices for each motor unit. This is the key used within this notebook, along with raw data, for Visualisation purposes.

__'SIL'__: Corresponding silhouette scores for each accepted source.

__PNR__: Corresponding pulse-to-noise ratio for each accepted source.

A simplified example of output of decomposed data can be seen below:

## Optional: save the output object

Cell below saves the output of the decomposition.

In [5]:
# decomp_sample = output 
# decomp_sample_pkl = open('decomp_sample_pkl.obj', 'wb') 
# pickle.dump(decomp_sample, decomp_sample_pkl)

Cell below loads the output saved above.

In [6]:
with open('decomp_sample_pkl.obj', 'rb') as f: output = pickle.load(f)

## Visualize results

Finally, run the cells below to get interactive dashboard of the decomposed data.

There are four parameters for the dashboard. _decomp\_results_ and _raw_ take in decomposition results and raw signal, respectively. You are unlikely to want to change it within the function, and most likely will select those earlier. However, if you want to plug in the data directly, you may use those parameters. The 'method' parameter allows to select the metric for calculating the average mismatch score between peak contribution and average MUAP shape. Currently, RMSE is the only available metric, but other metrics can be added easily in the future by adding functions to the viz.py script. The 'preset' parameter allows to select arrangements of the channel. Currently, you may select between 'standard' and 'vert63' arrangements. New arrangements may be added within the channel_preset() function in the viz.py script.

This will create a simple interactive dashboard. On top of the dashboard is the widget for selecting between motor unit action potentials (MUAPs.) Below the MUAP selection widget is the "Delete Selected Peak" button. Once a specific peak is selected, you may mark it as false positive and it will be removed from the pulse train and rerun the dashboard. Below the widgets, dashboard contains four charts:

#### Chart 1: Signal Strength + Firing Rate. 
The top chart displays signal strength along with the instanteneous firing rate at that point.   
This chart allows for interval control. Select the interval of interest and move it around, and observe the graphs below zoom in according to the selection.

#### Chart 2: Firing Rate. 
The second chart displays the instanteneous firing rate of each spike. Outliars may be indicative of false positive firing. Click on an individual data point to select corresponding pulse. This will allow to see the given firing's contribution to the MUAP shape, and delete it with the click of the button.

#### Chart 3: Signal Strength
The third chart displays signal strength of each firing. Click on an individual data point to select corresponding pulse. This will allow to see the given firing's contribution to the MUAP shape, and delete it with the click of the button.

#### Chart 4: MUAP Shapes + Peak Contributions
The last chart displays the average shape for each channel. The title displays the currently selected motor unit. If an individual peak has been selected using Chart 2 or 3, then an overlay will be displayed on each sub-chart that shows the contribution of the selected peak towards the shape for each channel. You may toggle the opacity of that overlay and the MUAP shape using the legend. Additionally, the peak firing index and RMSE will displayed in the title of the chart.

In [7]:
def select_peak(
    selection, mu_index, raw, shape_dict, pt, preset="standard", method=RMSE
):
    """
    Retrieves a given peak (if any) and re-graphs MUAP plot via muap_plot() function.
    Called within dashboard() function, binded to the peak selection on pulse graphs.

    Parameters
    ----------
        selection: selection object
            Selection object to dig into and retrieve peak index to plot.

        mu_index: int
            Currently plotted Motor Unit.

        raw: numpy.ndarray
            Raw EMG signal array.

        shape_dict: dict
            Dictionary containing MUAP shapes for each motor unit.

        pt: numpy.ndarray
            Multi-dimensional array containing indices of firing times
            for each motor unit.

    Returns
    -------
        altair plot object

    """
    global selected_peak

    if not selection:
        plot = muap_plot(shape_dict, mu_index, l=31, preset=preset, method=RMSE)
        selected_peak = -1

    else:
        selected_peak = selection[0] - 1
        # for some reason beyond my grast these are 1-indexed
        peak = pt[mu_index][selected_peak]

        peak_data = muap_dict_by_peak(raw, peak, mu_index=mu_index, l=31)
        plot = muap_plot(
            shape_dict,
            mu_index,
            peak_data,
            l=31,
            peak=str(peak),
            preset=preset,
            method=RMSE,
        )

    return pn.Column(
        pn.Row(
            pn.pane.Vega(plot, debounce=10, width=750),
        )
    )

In [8]:
def dashboard(decomp_results, raw, mu_index=0, preset="standard", method=RMSE):
    """
    Parent function for creating interactive visual component of decomposition.
    Dashboard consists of four plots:
    1. Plot of firings and signal, primarily for zooming and navigating.
    2. Plot of signal strength, which allows for peak selection.
    3. Plot of firings, which allows for peak selection.
    4. MUAP plot of individual motor unit shapes by channel, with selected peak overlay

    Parameters
    ----------
        decomp_results: dict
            Decomposition results.
            Must contain [MUPulses] key with the pulses array.

        raw: numpy.ndarray
            Raw EMG data.

        mu_index: int
            Currently plotted Motor Unit.

    Returns
    -------
        panel object containing interactive altair plots
    """
    # A little hacky, because I don't know how to pass params to the button
    # Delete button uses these to pass preset and method to muap_plot
    global gl_preset
    global gl_method
    gl_preset = preset
    gl_method = method

    signal = flatten_signal(raw)
    signal = np.apply_along_axis(
        butter_bandpass_filter,
        axis=1,
        arr=signal,
        lowcut=10,
        highcut=900,
        fs=2048,
        order=6,
    )
    centered = center_matrix(signal)
    c_sq = centered ** 2
    c_sq_mean = c_sq.mean(axis=0)

    pt = decomp_results["MUPulses"]

    shape_dict = muap_dict(raw, pt, l=31)
    pulse = pulse_plot(pt, c_sq_mean, mu_index, sel_type="interval")
    pulse_pn = pn.pane.Vega(pulse, debounce=10)
    mu_charts_pn = pn.bind(
        select_peak,
        pulse_pn.selection.param.sel_peak,
        mu_index,
        raw,
        shape_dict,
        pt,
        preset,
        method,
    )

    button_del = pn.widgets.Button(
        name="Delete Selected Peak", button_type="primary", width=50
    )
    button_del.on_click(b_click)

    res = pn.Column(
        button_del,
        pulse_pn,
        mu_charts_pn,
    )

    return res

In [14]:
def b_click(event):
    """
    Function triggered by clicking "Delete Selected Peak" button on the dashboard
    Bound to the button widget inside dashboard() function
    Deletes selected peak from the output variable and reruns the dashboard

    Parameters
    ----------
        event: event
            event that triggered the funciton

    Returns
    -------
        Null
    """
    if selected_peak > -1:

        # Get the peak and the selected MU index
        ###############################
        peak = dash_p[1][0][1].object.data.iloc[selected_peak]["Pulse"]
        mu_index = dash_p[0][0].value

        # Change decomp_results:
        ###############################
        global output
        output = remove_false_peak(output, mu_index, peak)

        # Reconstruct the plot:
        ###############################
        raw = raw_data_dict["SIG"]
        decomp_results = output
        signal = flatten_signal(raw)
        signal = np.apply_along_axis(
            butter_bandpass_filter,
            axis=1,
            arr=signal,
            lowcut=10,
            highcut=900,
            fs=2048,
            order=6,
        )
        centered = center_matrix(signal)
        c_sq = centered ** 2
        c_sq_mean = c_sq.mean(axis=0)
        pt = decomp_results["MUPulses"]
        shape_dict = muap_dict(raw, pt, l=31)
        pulse = pulse_plot(pt, c_sq_mean, mu_index, sel_type="interval")
        pulse_pn = pn.pane.Vega(pulse, debounce=10)
        dash_p[1][0][1] = pulse_pn

        # Also redo mu_charts graph so that it no longer selects the deleted peak:
        mu_charts_pn = pn.bind(
            select_peak,
            pulse_pn.selection.param.sel_peak,
            mu_index,
            raw,
            shape_dict,
            pt,
            preset=gl_preset,
            method=gl_method,
        )
        dash_p[1][0][2] = mu_charts_pn

mu_index_widget = pn.widgets.Select(name='Motor Unit:', options=[i for i in range(0, len(output["MUPulses"]))], value=0)

In [16]:
dash_p = interact(dashboard,
                  decomp_results=fixed(output), 
                  raw=fixed(raw_data_dict['SIG']),
                  mu_index=mu_index_widget,
                  preset=fixed('standard'),
                  method=fixed(RMSE))

dash_p

Column
    [0] Column
        [0] Select(name='Motor Unit:', options=[0, 1, 2, 3, 4, ...], value=0)
    [1] Row
        [0] Column
            [0] Button(button_type='primary', name='Delete Selected Peak', width=50)
            [1] Vega(VConcatChart, debounce=10, selection=Selection)
            [2] ParamFunction(function)

## Save the polished data

After manually removing false positive firing peaks, you may want to save the dictionary containing corrected firings. You can later reload it in the cell above. To save the cleaned up output, please run the cell below.

In [11]:
decomp_sample = output 
decomp_sample_pkl = open('decomp_sample_pkl.obj', 'wb') 
pickle.dump(decomp_sample, decomp_sample_pkl)